The purpose of the following code was to scrape articles from presseportal.de using BeautifulSoup.
Parts of this code were generated with the help of ChatGPT and altered for the specific needs of this study.

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime, timedelta
from urllib.parse import urljoin
import re
import time

base_url = "https://www.presseportal.de/pressemitteilungen"
site_root = "https://www.presseportal.de"

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122 Safari/537.36"
}

start_date = datetime.strptime("2020-01-01", "%Y-%m-%d")
end_date   = datetime.strptime("2023-12-31", "%Y-%m-%d")

STOP_RE = re.compile(r"(Rückfragen bitte an|Pressekontakt|Original-Content(?:\s*von)?|Kontakt:|news aktuell GmbH)", re.I)

def is_article_url(href: str) -> bool:
    if not href:
        return False
    return any(p in href for p in ("/pm/", "/nr/", "/story/", "/blaulicht/pm/"))

def cut_footer(text: str) -> str:
    m = STOP_RE.search(text)
    return text[:m.start()].strip() if m else text.strip()

def extract_text_and_topics(html: str):
    s = BeautifulSoup(html, "lxml")

    # text
    scope = s.find("article") or s.select_one('div[itemprop="articleBody"]') or s
    parts = [el.get_text(" ", strip=True) for el in scope.find_all(("p", "li"))]
    text = "\n\n".join([t for t in parts if t])
    m = STOP_RE.search(text)
    if m:
        text = text[:m.start()].strip()

    # topics
    topics = []
    heading = None
    for div in s.find_all("div", class_=lambda c: c and "thisisnoh" in c.split()):
        if div.get_text(strip=True) == "Themen in dieser Story":
            heading = div
            break

    ul = None
    if heading:
        sib = heading.next_sibling
        while sib and (getattr(sib, "name", None) is None):
            sib = sib.next_sibling
        if getattr(sib, "name", None) == "ul" and "tags" in (sib.get("class") or []):
            ul = sib

    if not ul:
        for cand in s.find_all("ul", class_="tags"):
            anchors = cand.find_all("a", href=True)
            if any(("/st/" in a["href"]) or ("/t/" in a["href"]) for a in anchors):
                ul = cand
                break

    if ul:
        anchors = ul.find_all("a", href=True)
        topics = [a.get_text(strip=True)
                  for a in anchors
                  if ("/st/" in a["href"]) or ("/t/" in a["href"])]

    return text, topics

def fetch_detail(session: requests.Session, url: str):
    try:
        r = session.get(url, headers=headers, timeout=20)
        if r.status_code != 200:
            return "", []
        return extract_text_and_topics(r.text)
    except Exception as e:
        print(f"  error fetching detail: {e}")
        return "", []

rows = []
stop = False

with requests.Session() as session:
    while not stop:
        for offset in range(0, 300, 30):
            list_url = f"{base_url}/{offset}?startDate={start_date:%Y-%m-%d}&endDate={end_date:%Y-%m-%d}"
            print(f"Scraping page: {list_url}")
            r = session.get(list_url, headers=headers, timeout=20)
            if r.status_code != 200:
                print(f"  failed with status {r.status_code}")
                continue

            soup = BeautifulSoup(r.text, "lxml")
            cards = soup.find_all("article", class_="news")

            for card in cards:
                d = card.find("div", class_="date")
                if not d:
                    continue
                dt_txt = d.get_text(strip=True).replace("–", "-")
                try:
                    dt = datetime.strptime(dt_txt, "%d.%m.%Y - %H:%M")
                except ValueError:
                    continue
                if dt < start_date:
                    print("Reached start date, stopping.")
                    stop = True
                    break

                cat_el = card.find("a", class_="news-topic")
                category = cat_el.get_text(strip=True) if cat_el else "N/A"
                h3 = card.find("h3")
                title = h3.get_text(strip=True) if h3 else "N/A"

                link = None
                if h3:
                    a = h3.find("a", href=True)
                    if a and is_article_url(a["href"]):
                        link = urljoin(site_root, a["href"])
                if not link:
                    a = card.find("a", href=True)
                    if a and is_article_url(a["href"]):
                        link = urljoin(site_root, a["href"])

                ps = card.find_all("p")
                source = ps[0].get_text(strip=True) if ps else "N/A"
                teaser = ps[1].get_text(strip=True) if len(ps) > 1 else ""

                text, topics = ("", [])
                if link:
                    text, topics = fetch_detail(session, link)

                if not text:
                    text = teaser if teaser else "N/A"

                rows.append([dt, category, title, source, text, link, topics])
                time.sleep(0.12)

            if stop:
                break

        df = pd.DataFrame(rows, columns=["date", "category", "title", "source", "text", "url", "topics"])
        min_date = df["date"].min()
        if pd.isna(min_date) or min_date < start_date:
            print("Reached earliest date. Stopping.")
            break
        end_date = max(start_date, min_date - timedelta(days=1))
        print(f"New end_date set to {end_date:%Y-%m-%d}")
        if end_date <= start_date:
            break

print(f"Total articles scraped: {len(df)}")

df.to_csv(r"master_presseportal_news_articles_dataset.csv",
          index=False, encoding="utf-8")


Scraping page: https://www.presseportal.de/pressemitteilungen/0?startDate=2020-01-01&endDate=2023-12-31
Scraping page: https://www.presseportal.de/pressemitteilungen/30?startDate=2020-01-01&endDate=2023-12-31
Scraping page: https://www.presseportal.de/pressemitteilungen/60?startDate=2020-01-01&endDate=2023-12-31
Scraping page: https://www.presseportal.de/pressemitteilungen/90?startDate=2020-01-01&endDate=2023-12-31
Scraping page: https://www.presseportal.de/pressemitteilungen/120?startDate=2020-01-01&endDate=2023-12-31
Scraping page: https://www.presseportal.de/pressemitteilungen/150?startDate=2020-01-01&endDate=2023-12-31
Scraping page: https://www.presseportal.de/pressemitteilungen/180?startDate=2020-01-01&endDate=2023-12-31
Scraping page: https://www.presseportal.de/pressemitteilungen/210?startDate=2020-01-01&endDate=2023-12-31
Scraping page: https://www.presseportal.de/pressemitteilungen/240?startDate=2020-01-01&endDate=2023-12-31
Scraping page: https://www.presseportal.de/pressemi